In [1]:
# 模块和包都是在逐步的探索中所需要的，全部汇总到这里，
#    并不是一开始就知道了 ^_^ ^_^ ^_^ 
# 不熟悉的模块和包，强烈建议查看官方文档说明以及例子
# 1、导入模块和包
import pandas as pd    # 加载并处理csv文件
import datetime        # 利用datetime处理时间戳
# cPickle 数据以二进制进行高效的储存到文件
import _pickle as cPickle 
# defaultdict 设置稀疏矩阵的 NULL 位置的默认值
from collections import defaultdict 
# 利用scipy sparse 构建稀疏矩阵
import scipy.sparse as ss     
import scipy.io as sio    # 利用scipy储存评分矩阵
# 利用numpy创建指定长度或形状的矩阵以及矩阵运算
import numpy as np 
# numpy.random中的randn函数生成正态分布的随机数据
from numpy.random import random    
import time    # 计算训练时迭代的时间
import json    # 将模型参数保存和加载 json文件
import scipy    # 将储存加载的稀疏评分矩阵转换为numpy矩阵

In [22]:
# 加载训练集数据
data_path = "./../dataset/BX-CSV-Dump/"
train = pd.read_csv(data_path + "train.csv", sep = ",", nrows = 2000 )
# 计算用户数量和电影数量
unique_user = train['User-ID'].unique()
unique_item = train['ISBN'].unique()
user_num = unique_user.shape[0]
item_num = unique_item.shape[0]
print("the number of user:",user_num)
print("the number of movie:",item_num)

the number of user: 1511
the number of movie: 1919


In [23]:
# 4、模型探索
# 4.1、BiasSVD r(ui) = average + bu + bi + Pu'T Qi
# 4.1.1、初始化BiasSVD模型参数
# 隐含变量的维数 m*n = m*k * k*n
# 设置参数变量，可以随时调整模型参数值
k = 12
# Item和User的偏置项
# zeros和ones分别可以创建指定长度或形状的全0或全1数组。
bi = np.zeros((item_num, 1))    # n 行 1 列
bu = np.zeros((user_num, 1))    # m 行 1 列
# Item和User的隐含向量
qi = np.zeros((item_num, k))    # n 行 k 列    
pu = np.zeros((user_num, k))    # m 行 k 列
# 使用numpy.random中的randn函数生成一些正态分布的随机数据
# pu和qi，两个矩阵的初始化，可以使全为零
# 也可以符合正态分布（高斯分布），同时也隐向量维度k的平方有关
for user_id in range(user_num):
    pu[user_id] = np.reshape(random((k, 1)) / 10 * np.sqrt(k), k)
       
for item_id in range(item_num):
    qi[item_id] = np.reshape(random((k, 1)) / 10 * np.sqrt(k), k)

# 所有用户的平均打分
average = train['Book-Rating'].mean()

In [24]:
# 4.1.2、根据初始化BiasSVD模型参数进行预测评分
# 根据当前模型参数，预测用户对物品打分
def BiasSVD_prediction(user_id, item_id):  
    score = average + bi[item_id] + bu[user_id] + np.sum(qi[item_id] * pu[user_id]) 
    return score 

In [25]:
# 加载重新索引的用户索引表
user_index = cPickle.load(open(data_path + "user_index.pkl", 'rb'))
# 加载重新索引的电影索引表
item_index = cPickle.load(open(data_path + "item_index.pkl", 'rb'))

In [26]:
# 4.1.3、BiasSVD模型训练
# 参数：gamma —学习率、Lambda —正则参数、steps —迭代次数
# 初始化BiasSVD模型超参数
steps = 20
each_rmse_list = list()
gamma = 0.005
each_gamma_list = list()
Lambda = 0.15
# 总的评分记录数目
total_records = train.shape[0]
# 第一次迭代时间开始
time_start = time.time()
print("开始进行{}个step的训练".format(steps))
each_time_start = time_start

for step in range(steps):
    print('The {}-th  step is running'.format(step))
    rmse_sum = 0.0    # 真实评分与预测评分的差值的平方            
    # 利用numpy.random.permutation函数可以轻松实现
    #   对Series或DataFrame的列的排列工作，随机重排序
    # 通过需要排列的轴的长度调用permutation， 
    #   可产生一个表示新顺序的整数数组
    # 将训练样本打散顺序 
    total_num = np.random.permutation(total_records)  
    for index in range(total_records):
        # 每次一个训练样本,即一条记录
        row = total_num[index]
        
        user_id = user_index[ train.iloc [row] ['User-ID'] ]
        item_id = item_index[ train.iloc [row] ['ISBN'] ]
        rating = train.iloc[row]['Book-Rating']
        
        # 目标函数构建
        # 预测残差
        eui = rating - BiasSVD_prediction(user_id, item_id)
        # residual sum of squares 残差平方和 
        rmse_sum += eui**2

        # 随机梯度下降，更新
        bu[user_id] += gamma * (eui - Lambda * bu[user_id])  
        bi[item_id] += gamma * (eui - Lambda * bi[item_id])
                
        temp = qi[item_id]  
        qi[item_id] += gamma * (eui * pu[user_id] - Lambda * qi[item_id])
        pu[user_id] += gamma * (eui * temp - Lambda * pu[user_id])
            
    # 学习率递减 / learning rate improve 3th
    each_gamma_list.append(gamma)
    gamma = gamma * 1.2
    #gamma = gamma * 0.93
    # Root Mean Square Error RMSE
    each_rmse = np.sqrt(rmse_sum / total_records)
    each_rmse_list.append(each_rmse)
    
    # 每次迭代时间结束
    each_time_tick = time.time()
    # 每次迭代消耗的时间
    each_cost_time = each_time_tick - each_time_start
    # 更新计算每次迭代的时间
    each_time_start = each_time_tick
    print("完成第{}个step的训练, each_rmse={}, 耗时{:.4f}秒".format(
        step + 1, each_rmse, each_cost_time))

# 计算训练数据集消耗的总时间
time_end = time.time()
total_cost_time = time_end - time_start
print("结束了{}个step的训练，总耗时{:.4f}秒".format(steps, total_cost_time))

开始进行20个step的训练
The 0-th  step is running
完成第1个step的训练, each_rmse=[3.84621997], 耗时3.1902秒
The 1-th  step is running
完成第2个step的训练, each_rmse=[3.77697879], 耗时3.0772秒
The 2-th  step is running
完成第3个step的训练, each_rmse=[3.69737191], 耗时3.1142秒
The 3-th  step is running
完成第4个step的训练, each_rmse=[3.606014], 耗时3.1472秒
The 4-th  step is running
完成第5个step的训练, each_rmse=[3.50113257], 耗时3.0892秒
The 5-th  step is running
完成第6个step的训练, each_rmse=[3.38072879], 耗时3.1082秒
The 6-th  step is running
完成第7个step的训练, each_rmse=[3.2419778], 耗时3.2212秒
The 7-th  step is running
完成第8个step的训练, each_rmse=[3.08127298], 耗时3.1422秒
The 8-th  step is running
完成第9个step的训练, each_rmse=[2.89384913], 耗时3.0752秒
The 9-th  step is running
完成第10个step的训练, each_rmse=[2.6743621], 耗时3.1202秒
The 10-th  step is running
完成第11个step的训练, each_rmse=[2.41916279], 耗时3.1132秒
The 11-th  step is running
完成第12个step的训练, each_rmse=[2.12875911], 耗时3.1022秒
The 12-th  step is running
完成第13个step的训练, each_rmse=[1.80947542], 耗时3.0892秒
The 13-th  step is r

In [27]:
# 4.1.4、BiasSVD模型参数保存
# 模型参数保存为json， 键（模型参数）和值（模型参数值）
def BiasSVD_parameter_toJson(filepath):
    BiasSVD_parameter_dict = dict()
    BiasSVD_parameter_dict['average'] = average
    BiasSVD_parameter_dict['k'] = k
    
    # numpy.ndarray.map() 将列表转换为矩阵
    # numpy.ndarray.tolist() 将矩阵转换为列表
    BiasSVD_parameter_dict['bi'] = bi.tolist()
    BiasSVD_parameter_dict['bu'] = bu.tolist()
    
    BiasSVD_parameter_dict['qi'] = qi.tolist()
    BiasSVD_parameter_dict['pu'] = pu.tolist()

    # json.loads即可将JSON字符串转换成Python形式
    # json.dumps则将Python对象转换成JSON格式
    BiasSVD_parameter_toJsonTxt = json.dumps(BiasSVD_parameter_dict)
    with open(filepath, 'w') as file:
        file.write(BiasSVD_parameter_toJsonTxt)


In [28]:
# 4.1.5、BiasSVD模型参数加载
def BiasSVD_parameter_load_fromJson(filepath):
    with open(filepath, 'r') as file:
        BiasSVD_parameter_dict = json.load(file)

        average = BiasSVD_parameter_dict['average']
        K = BiasSVD_parameter_dict['k']

        # numpy.asarray 将数据转化为 ndarray 格式
        bi = np.asarray(BiasSVD_parameter_dict['bi'])
        bu = np.asarray(BiasSVD_parameter_dict['bu'])
    
        qi = np.asarray(BiasSVD_parameter_dict['qi'])
        pu = np.asarray(BiasSVD_parameter_dict['pu'])


In [29]:
# 4.1.4、BiasSVD模型参数保存
BiasSVD_parameter_toJson(data_path + 'BiasSVD_modelParameter.json')
# 4.1.5、BiasSVD模型参数加载
BiasSVD_parameter_load_fromJson(data_path + 'BiasSVD_modelParameter.json')

In [30]:
# 4.1.6、BiasSVD模型评测指标
# 数据集的总的样本空间数 T
# 真实的用户对物品的评分 rui
# 预测的用户对物品的评分 r`ui
# 真实评分与预测评分的差值 rss = rui - r`ui

# 对 rss 进行绝对值得到 rss_absolute
# 对 rss_absolute 进行求和得到 rss_absolute_sum
# MAE = rss_absolute_sum / T

# 对 rss 进行平方得到 rss_square
# 对 rss_square 进行求和得到 rss_square_sum
# RMSE = rss_square_sum / T
# MSE = rss_square / T

def calculate_MAE(dataset):
    # 真实评分与预测评分的差值的绝对值之和
    mae_sum = 0.0
    T = dataset.shape[0]
    total_num = np.random.permutation(T)
    for index in range(T):
        # 每次计算一个样本,即一条记录
        row = total_num[index]
        rating = dataset.iloc[row]['Book-Rating']
        
        # 预测残差
        eui = rating - BiasSVD_prediction(user_id, item_id)
        # residual sum of squares 残差平方和 
        mae_sum += abs(eui)
            
    # Root Mean Square Error RMSE
    MAE = mae_sum / total_records
    return MAE
    

def calculate_RMSE(dataset):
    # 真实评分与预测评分的差值的平方和
    rmse_sum = 0.0    
    T = dataset.shape[0]
    total_num = np.random.permutation(T)
    for index in range(T):
        # 每次计算一个样本,即一条记录
        row = total_num[index]
        rating = dataset.iloc[row]['Book-Rating']
        
        # 预测残差
        eui = rating - BiasSVD_prediction(user_id, item_id)
        # residual sum of squares 残差平方和 
        rmse_sum += eui**2
            
    # Root Mean Square Error RMSE
    RMSE = np.sqrt(rmse_sum / total_records)
    return RMSE


In [31]:
# 训练集评测指标：RMSE、MAE
print("the MAE of the RS of the train set :", calculate_MAE(train))
print("the RMSE of the RS of the train set :", calculate_RMSE(train))

the MAE of the RS of the train set : [6.23036162]
the RMSE of the RS of the train set : [7.17357198]
